In [92]:
from bs4 import BeautifulSoup as bs
from requests_html import HTMLSession
import pandas as pd
from IPython.display import clear_output
import time

In [68]:
#get social profile from etherscan
#return dictionary of social profile
def get_social_profiles(token):
    URL_base = "https://etherscan.io/token/"
    URL_complete = URL_base + token
    session = HTMLSession()
    page = session.get(URL_complete)
    soup = bs(page.content,"html.parser")
    results = soup.find(id="content")
    header = results.find("h2",string = "Profile Summary")
    profile_card = header.parent.parent.parent
    social_profiles = profile_card.find_all("li")
    social_profiles_map = {}
    social_profiles_map["token"] = token
    for social_profile in social_profiles:
        title_list = social_profile.find("a")["data-original-title"].split(": ")
        href = social_profile.find("a")["href"]
        title = title_list[0]
        if "Email" == title:
            href = title_list[1]
        social_profiles_map[title] = [href]
    return social_profiles_map



In [86]:
token_list = []

In [94]:
track_int = 0
social_profiles_df = pd.DataFrame()
social_profiles_df['token'] = pd.NaT
wait = True
for token in token_list:
    wait = True
    while(wait):
        clear_output(wait=True)
        print(token)
        print(f"complete {track_int/2485 * 100}%")
        try:
            social_profiles = get_social_profiles(token)
            wait=False
            social_profiles_df = pd.concat([social_profiles_df, pd.DataFrame.from_dict(social_profiles)],ignore_index=True)
            track_int+=1
        except:
            wait=True
            print("wating...")
            time.sleep(2)
    

0xb548daa2aebeebe661768a6dfa98c0649c670385
complete 99.95975855130784%


In [95]:
social_profiles_df.to_pickle("token_social_profile.pkl")

In [101]:
total = social_profiles_df.shape[0]
col_names = social_profiles_df.columns[1:]


In [103]:
social_profiles_df[col_names[0]].isna().sum()

489

In [109]:
for col_name in col_names:
    missing_count = social_profiles_df[col_name].isna().sum()
    print(f"{col_name} missing {round(missing_count/total * 100,2)}%")

Email missing 19.68%
Twitter missing 0.0%
Discord missing 71.67%
Opensea missing 90.34%
Blog missing 47.04%
Github missing 53.08%
Telegram missing 25.96%
Whitepaper missing 44.63%
CoinMarketCap missing 71.71%
CoinGecko missing 73.16%
Reddit missing 72.19%
Facebook missing 55.09%
Linkedin missing 79.6%
Slack missing 93.76%
Bitcointalk missing 79.36%
Uniswap missing 93.12%
Ticketing missing 98.67%
WeChat missing 99.4%
